# COMP9318 Lab3

## Instructions
1. This note book contains instructions for **COMP9318-Lab3**.

* You are required to complete your implementation in a file `submission.py` provided along with this notebook.

* You are not allowed to print out unnecessary stuff. We will not consider any output printed out on the screen. All results should be returned in appropriate data structures via corresponding functions.

* You can submit your implementation for **Lab3** via following link: https://kg.cse.unsw.edu.au/submit/ .

* For each question, we have provided you with detailed instructions along with question headings. In case of any problem, you can post your query @ Ed.

* You are allowed to add other functions and/or import modules (you may have to in this lab), but you are not allowed to define global variables. **Only functions are allowed** in `submission.py`. 

* You should not import unnecessary modules/libraries, failing to import such modules at test time will lead to errors.

* We will provide immediate feedback on your submission. You can access your scores using the online submission portal on the same day. 

* For **Final Evaluation** we will be using a different dataset, so your final scores may vary.  

* You are allowed to submit as many times as you want before the deadline, but **ONLY the latest version will be kept and marked**.

* Submission deadline for this assignment is **20:59:59 on 2nd April, 2021 (SYDNEY TIME)**. We will **not** accept any late submissions.

# Question-1: Text Classification using Multinomial Naive Bayes

You are required to implement a multinomial naive bayes classifier to predict spam SMS.

The training data is a set of SMS categoried into `spam` and `ham`.

In [1]:
import pandas as pd

raw_data = pd.read_csv('./asset/data.txt', sep='\t')
raw_data.head()

,category,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In order to implement a unigram model, first we tokenize the text. We use the count corresponding to each token (word) in the SMS as its feature (i.e., bag of words). We store the features and catrgorical information for each SMS in a `dictionary`.

In [2]:
def tokenize(sms):
    return sms.split(' ')

def get_freq_of_tokens(sms):
    tokens = {}
    for token in tokenize(sms):
        if token not in tokens:
            tokens[token] = 1
        else:
            tokens[token] += 1
    return tokens

training_data = []
for index in range(len(raw_data)):
    training_data.append((get_freq_of_tokens(raw_data.iloc[index].text), raw_data.iloc[index].category))

For this lab, you need to **implement** a multinomial naive bayes classifier (i.e., `multinomial_nb()` in the file: `submission.py`) with add-1 smoothing. The input arguments of `multinomial_nb()` are:
* `training_data`: pre-processed data stored as a `dictionary`
* `sms`: test-sms (i.e., a list of tokens) that you need to categorize as `spam` and/or `ham`

The return value of `multinomial_nb()` should be the **ratio** of the probability of sms is spam and the probability of sms is ham. A return value larger than 1 implies the `sms` is spam and vice versa.

For example, a sample output is shown in the cell given below:

In [34]:
def multinomial_nb(training_data, sms):# do not change the heading of the function

    # ratio = the probability of sms is spam / the probability of sms is ham.
    # ratio = pro_spam_sms / pro_ham_sms

    # pro_spam = # of spam / total # of training data
    # pro_spam = # of spam / total # of training data
    count_spam = 0
    count_ham = 0
    training_token_spam = {} # {word, freq}
    training_token_ham = {} # {word, freq}
    vacabulary = {}
        
    for data in training_data:
        if data[1] == "spam":
            count_spam += 1
            for key, val in data[0].items():
                if key not in vacabulary:
                    vacabulary[key] = 1

                if key not in training_token_spam:
                    training_token_spam[key] = val
                else:
                    training_token_spam[key] += val
                    
        elif data[1] == "ham":
            count_ham += 1
            for key, val in data[0].items():
                if key not in vacabulary:
                    vacabulary[key] = 1
                    
                if key not in training_token_ham:
                    training_token_ham[key] = val
                else:
                    training_token_ham[key] += val

    total_distinct_words_in_spam = sum(training_token_spam.values())
    total_distinct_words_in_ham = sum(training_token_ham.values())
    
    pro_word_spam_dic = {}
    pro_word_ham_dic = {}
    for key, val in vacabulary.items():
        if key not in training_token_spam:
            pro_word_spam_dic[key] = (0+1) / (total_distinct_words_in_spam + len(vacabulary))
        else:
            pro_word_spam_dic[key] = (training_token_spam[key]+1) / (total_distinct_words_in_spam + len(vacabulary))

        if key not in training_token_ham:
            pro_word_ham_dic[key] = (0+1) / (total_distinct_words_in_ham + len(vacabulary))
        else:
            pro_word_ham_dic[key] = (training_token_ham[key]+1) / (total_distinct_words_in_ham + len(vacabulary))

    pro_spam = count_spam / len(training_data)
    pro_ham = count_ham / len(training_data)


    # pro_spam_sms = pro_sms_spam*pro_spam
    pro_spam_sms = pro_spam
    for word in sms:
        if word not in pro_word_spam_dic:
            continue
        else:
            pro_spam_sms = pro_spam_sms*pro_word_spam_dic[word]

    # pro_ham_sms = pro_sms_ham*pro_ham
    pro_ham_sms = pro_ham
    for word in sms:
        if word not in pro_word_ham_dic:
            continue
        else:
            pro_ham_sms = pro_ham_sms*pro_word_ham_dic[word]

    ratio = pro_spam_sms / pro_ham_sms
    
    return ratio 

In [35]:
## How we test your implementation...
# import submission_ans as submission

sms = 'I am not spam'
print(multinomial_nb(training_data, tokenize(sms)))
# print(submission.multinomial_nb(training_data, tokenize(sms)))

0.23427672955974846




# Test Environment

For testing, we have pre-installed the requisite modules and/or libraries in the testing environment. You are only allowed to use following libraries:
* python: 3.6.5
* pandas: 0.19.2

NOTE: You are required to implement the classifier by yourself. You are not allowed to import **sklearn** and/or any other library in Lab3.